# 用numpy实现RNN
本文针对上一个笔记[BPTT算法数学推导](./BackPropagation Through Time.ipynb)，用numpy实现了一个RNN，部分代码参考自wildML，在其基础上进行了包装，并融入了自己的一些理解，力求写的通俗易懂。  


### 读取语料库
首先是读取语料库的函数，这里直接摘抄自wildML的代码，这段代码写的十分精巧，将许多功能浓缩在一行，实在是令我叹为观止。

In [ ]:
'''
读取语料库
'''
def load_corpus():
    print '='*50
    print 'Loading CSV file...'
    with open('data/reddit-comments-2015-08.csv', 'rb') as f:
        reader = csv.reader(f)
        # 跳过开头的'body'
        reader.next()
        # 将整篇文档分成句子的列表
        sentences = itertools.chain(*[nltk.sent_tokenize(x[0].decode('utf-8').lower()) for x in reader])
        # 为每句话加上 SENTENCE_START 和 SENTENCE_END 符号
        sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
    print 'Parsed %d sentences.'%len(sentences)
    return sentences

将文档拆成句子列表的这一句话写的非常geek，也非常难懂
```
sentences = itertools.chain(*[nltk.sent_tokenize(x[0].decode('utf-8').lower()) for x in reader])
```
我来分析一下这句话干了哪些事情：
1. 调用nltk的`sent_tokenize`对段落x[0]分句，将其转变为句子的列表，x[0]表示csv文件里一行的内容
2. 用列表生成式遍历csv文件每一行，生成句子列表的嵌套列表
3. 用星号`*`对这个嵌套列表解包，将其变为一个一个的列表
4. 用`chain()`函数将这些列表合并为一个列表  

`chain()`函数的定义如下：

In [ ]:
def chain(*iterables):
    # chain('ABC', 'DEF') --> A B C D E F
    for it in iterables:
        for element in it:
            yield element

它的作用是将多个迭代器作为参数, 但只返回单个迭代器, 它产生所有参数迭代器的内容, 就好像他们是来自于一个单一的序列。  
最后再用一个列表生成式为每一句话加上起始符和结束符。
```
sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
```

### 文本的预处理
文本预处理主要完成了：
1. 对句子的分词
2. 统计词频
3. 获取高频词，建立索引
4. 替换未登录词

In [ ]:
'''
文本预处理
'''
def preprocessing(sentences):
    print '='*50
    print 'Preprocessing...'
    # 对每个句子进行分词
    tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]
    # 统计词频
    word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
    print "Found %d unique words tokens." % len(word_freq.items())
    # 获得词频最高的词，并建立索引到词、词到索引的向量
    vocab = word_freq.most_common(vocabulary_size-1)
    index_to_word = [x[0] for x in vocab]
    index_to_word.append(unknown_token)
    word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

    print "Using vocabulary size %d." % vocabulary_size
    print "The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1])                 
    # 将未登录词替换为unknown token
    for i, sent in enumerate(tokenized_sentences):
        tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]
    return tokenized_sentences, word_to_index

1. 对句子分词
`nltk.word_tokenize(sent)`作用是对输入的句子进行分词，输出为分词结果组成的列表。`tokenized_sentences`是一个嵌套列表，列表中的每一个元素表示一条句子的分词结果。
2. 统计词频
首先用`*`将`tokenized_sentences`分解为多个list，接着调用`itertools.chain()`函数将这些list接合成一个list，作为`nltk.FreqDist()`函数的输入，返回一个字典，键为word，值为该word的词频。
3. 获取高频词，建立索引
用`most_common`函数获取频率最高的`vocabulary_size-1`个词，返回值`vocab`是一个tuple的list，其格式如下：
```
[(',', 18713), ('the', 13721), ('.', 6862), ('of', 6536), ('and', 6024),
('a', 4569), ('to', 4542), (';', 4072), ('in', 3916), ('that', 2982)]
```
接着我们遍历这个list，将高频词从每个tuple取出来单独构成一个list，并在最后加入unknown_token表示未登录词：
```
index_to_word = [x[0] for x in vocab]
```
然后我们要建立词的索引，`enumerate()`函数接收一个list，并为list中的每个元素生成一个序号。一种较为naive的写法是

In [ ]:
i = 0
for item in iterable:
    print i, item
    i += 1

而使用enumerate我们可以将代码简化为：

In [ ]:
for i, item in enumerate(iterable):
    print i, item

接着我们生成一个由词和索引构成的tuple组成的list，并用它生成一个dict `word_to_index`。

4.替换未登录词
这里使用了一种较为高级的列表生成式：
```
[w if w in word_to_index else unknown_token for w in sent]
```
在生成列表时候，我们可以加入条件判断以完成复杂的表达式赋值，这里如果w出现在索引字典中则保持不变，否则将被替换为unknown_token。

### 生成训练数据
上一步中，我们获得了分词结果`tokenized_sentences`和词索引`word_to_index`，这一节我们利用它们来生成训练集。

In [ ]:
'''
生成训练数据
'''
def gen_train_data():
    # 读取语料库
    sentences = load_corpus()
    # 预处理
    tokenized_sentences, word_to_index = preprocessing(sentences)
    # 创建训练数据
    X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
    y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])    
    return X_train, y_train

`gen_train_data()`函数首先读取语料库，接着对句子进行了预处理，最后生成训练数据集。`X_train`和`y_train`都是二维numpy数组，在初始化这两个数组时我们采用了含有两层嵌套循环的列表生成式：

In [ ]:
    X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
    y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])    

第一层循环遍历每一条句子的分词序列，第二层循环遍历序列中的每个词将它们转化为索引。
注意到`sent[:-1]`代表`[sentence_start_token x ]`，`sent[1:]`代表`[x sentence_end_token]`，这是由于我们这个任务是训练一个语言模型，每次预测下一个出现的词，因此`y_train`中每个词对应`X_train`中下一个出现的词，即$y_t=x_{t+1}$。

### 实现softmax函数

In [ ]:
def softmax(x):
    xt = np.exp(x - np.max(x))
    return xt / np.sum(xt)

解释一下为什么要减去max(x)。由于x可能会很大，exp以后可能会上溢出（overflow），而softmax是一个比值，因此如果我们在分子分母上同时乘/除以一个常数是不会影响最终结果的，我们只要同时除以一个较大的数就可以防止overflow。显然这里除以最大值是最好的，因为我们无法确定x的数量级，而最大值是关于x数量级自适应增长的。 